## NYC NTA table for bucketed time away from home by day
##### Output is large Excel table with all NTAs by day, total devices counted for bucketed_time_away, devices away for more than >1321 mins (i.e. 22 hours), devices away for < 20 min, and calculated share of devices away from home for >1321 min of devices counted in bucketed time away

In [1]:
import pandas as pd
import numpy as np
import s3fs
import os
import time

In [2]:
from geo import stco,sub
from safegraph_py_functions import safegraph_py_functions as sgpy

In [3]:
%load_ext dotenv
%dotenv
myAccessKey = os.getenv('myAccessKey')
mySecretKey = os.getenv('mySecretKey')

start = time.time()

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
# read nyc origin cbgs & geoxwalk doc
cbg_nyc = pd.read_csv(f'../data/nyc_cbg.csv')
nta_nyc = pd.read_excel(f'../data/nyc_geo_xwalk.xlsx')

In [6]:
# specify the SG key and secret
fs = s3fs.S3FileSystem(profile="safegraphws", key=myAccessKey, secret=mySecretKey, client_kwargs={'endpoint_url': 'https://s3.wasabisys.com', 'region_name':'us-east-1'})

## RUNNING FOR 2019 ONLY

In [7]:
y = "2019"
#years = ["2019","2020"]

monthList =["01","02","03","04","05","06","07","08"]
dayNumList =[31, 28, 31, 30, 31, 30, 31,31] 
dayList =["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]

In [8]:
#Run an iteration to unpack the bucketed time away from home

In [9]:
frames = [] 
for m in range(0,8):
    for d in range(0,dayNumList[m]):
        with fs.open(f'sg-c19-response/social-distancing/v2/{y}/{monthList[m]}/{dayList[d]}/{y}-{monthList[m]}-{dayList[d]}-social-distancing.csv.gz','rb') as f:
            print(f'{y}-{monthList[m]}-{dayList[d]}')
            # read SG's file
            df = pd.read_csv(f, escapechar='\\', compression='gzip')
            # filter NYC's Origin CBGs
            df = pd.merge(nta_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")

            #unpack json bucketed time away from home
            df = sgpy.unpack_json_and_merge(df, json_column='bucketed_away_from_home_time', key_col_name='away_from_home_time', value_col_name='away_count')
            
            ##Make new columns
            df['date_y-m-d'] = df['date_range_start'].str[:10]
            #Make new table with select columns
            dff = df[['date_y-m-d','orig_nta','away_from_home_time','away_count']]
            frames.append(dff) 

2019-01-01
2019-01-02
2019-01-03
2019-01-04
2019-01-05
2019-01-06
2019-01-07
2019-01-08
2019-01-09
2019-01-10
2019-01-11
2019-01-12
2019-01-13
2019-01-14
2019-01-15
2019-01-16
2019-01-17
2019-01-18
2019-01-19
2019-01-20
2019-01-21
2019-01-22
2019-01-23
2019-01-24
2019-01-25
2019-01-26
2019-01-27
2019-01-28
2019-01-29
2019-01-30
2019-01-31
2019-02-01
2019-02-02
2019-02-03
2019-02-04
2019-02-05
2019-02-06
2019-02-07
2019-02-08
2019-02-09
2019-02-10
2019-02-11
2019-02-12
2019-02-13
2019-02-14
2019-02-15
2019-02-16
2019-02-17
2019-02-18
2019-02-19
2019-02-20
2019-02-21
2019-02-22
2019-02-23
2019-02-24
2019-02-25
2019-02-26
2019-02-27
2019-02-28
2019-03-01
2019-03-02
2019-03-03
2019-03-04
2019-03-05
2019-03-06
2019-03-07
2019-03-08
2019-03-09
2019-03-10
2019-03-11
2019-03-12
2019-03-13
2019-03-14
2019-03-15
2019-03-16
2019-03-17
2019-03-18
2019-03-19
2019-03-20
2019-03-21
2019-03-22
2019-03-23
2019-03-24
2019-03-25
2019-03-26
2019-03-27
2019-03-28
2019-03-29
2019-03-30
2019-03-31
2019-04-01

In [10]:
df_away = pd.concat(frames)
df_away = pd.pivot_table(df_away,values=['away_count'],index=['date_y-m-d','orig_nta'],columns=['away_from_home_time'],aggfunc=np.sum,fill_value=0,margins=True)

In [11]:
df_away.columns = df_away.columns.get_level_values(1)
df_away = df_away.reset_index()

In [12]:
df_away_reduced = df_away[['date_y-m-d', 'orig_nta','All','1321-1440','<20']]
df_away_reduced.loc[:,'1321-1440']/df_away_reduced.loc[:,'All'] = df_away_reduced['pct_away_1321-1440']

SyntaxError: cannot assign to operator (<ipython-input-12-64b4b52fd1ae>, line 2)

In [ ]:
df_away_reduced.to_excel(f'output/dara/Time Away/2019_time-away-reduced.xlsx')

In [ ]:
end = time.time()
elapsed = end - start
print(f'Run time - {elapsed} seconds')

In [ ]:
df_away_reduced.head(20)